In [2]:
import dlib
import numpy as np
import cv2
from mtcnn import MTCNN
import pandas as pd
import os
import shutil
import random
from PIL import Image
from datasets import Dataset, DatasetDict

In [3]:
def dlib_detector(img):
    dlib_face_detector = dlib.get_frontal_face_detector()
    dlib_face_locations = dlib_face_detector(img)
    return dlib_face_locations  # xmin,ymin,xmax,ymax

def mtcnn_detector_dlib(img):
    face_detector = MTCNN()
    face_info = face_detector.detect_faces(img)
    dlib_format = []
    for i in face_info:
        x, y, width, height = i["box"]
        dlib_format.append(dlib.rectangle(x,y,x+width,y+height))
    return dlib_format

In [4]:
def encodings(img,face_locations,pose_predictor,face_encoder):
    predictors = [pose_predictor(img, face_location) for face_location in face_locations]
    return np.array(face_encoder.compute_face_descriptor(img, predictors[0], 1))


pose_predictor = dlib.shape_predictor('Dlib_model/shape_predictor_68_face_landmarks.dat')
face_encoder = dlib.face_recognition_model_v1('Dlib_model/dlib_face_recognition_resnet_model_v1.dat')

In [ ]:
%%time
img = cv2.imread("images/single_face.jpg")

face_locations = dlib_detector(img)
face_encodings = encodings(img,face_locations,pose_predictor,face_encoder)
print(face_encodings)

In [7]:
import os
import shutil
import random

# Paths
input_dir = './Dataset_raw/original images/'
output_dir = './Dataset_organised/'

# Define split ratios (can be changed as needed)
train_split = 0.8
val_split = 0.1
test_split = 0.1

# Ensure the output directories exist
splits = ['train', 'val', 'test']
for split in splits:
    for class_name in os.listdir(input_dir):
        split_dir = os.path.join(output_dir, split, class_name)
        os.makedirs(split_dir, exist_ok=True)

# Process each folder (person1, person2, etc.)
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    
    if not os.path.isdir(class_path):
        continue  # Skip files, only process directories (like person1, person2)
    
    # Get all image files in the current class folder
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    random.shuffle(images)  # Shuffle to ensure randomness

    # Split images into train, val, and test
    total_images = len(images)
    train_count = int(total_images * train_split)
    val_count = int(total_images * val_split)
    
    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    # Copy files to train, val, and test directories
    for image_set, split in zip([train_images, val_images, test_images], splits):
        for image_name in image_set:
            src_path = os.path.join(class_path, image_name)
            dst_path = os.path.join(output_dir, split, class_name, image_name)
            
            try:
                shutil.copy(src_path, dst_path)
            except Exception as e:
                print(f"Error copying {src_path} to {dst_path}: {e}")

print("Dataset successfully organized!")


Dataset successfully organized!


# Paths to models and directories
pose_predictor = dlib.shape_predictor('Dlib_model/shape_predictor_68_face_landmarks.dat')
face_encoder = dlib.face_recognition_model_v1('Dlib_model/dlib_face_recognition_resnet_model_v1.dat')
input_dir = './Dataset_raw/original images/'
output_dir = './Dataset_organised/'

# Define split ratios (train, validation, test)
train_split = 0.8
val_split = 0.1
test_split = 0.1

# Ensure the output directories exist
splits = ['train', 'val', 'test']
for split in splits:
    for class_name in os.listdir(input_dir):
        split_dir = os.path.join(output_dir, split, class_name)
        os.makedirs(split_dir, exist_ok=True)

# Function to extract embeddings
def extract_embeddings(img_path):
    img = Image.open(img_path)
    img = np.array(img)
    
    face_locations = dlib_detector(img)
    if not face_locations:
        return None  # No faces detected
    
    # Extract embeddings for detected faces
    embeddings = encodings(img, face_locations, pose_predictor, face_encoder)
    return embeddings

# Process each folder (person1, person2, etc.)
dataset_dict = {'train': [], 'val': [], 'test': []}
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    
    if not os.path.isdir(class_path):
        continue  # Skip files, only process directories (like person1, person2)
    
    # Get all image files in the current class folder
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    random.shuffle(images)  # Shuffle to ensure randomness

    # Split images into train, val, and test
    total_images = len(images)
    train_count = int(total_images * train_split)
    val_count = int(total_images * val_split)
    
    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    # Add images to train, val, and test splits with embeddings
    for split, image_set in zip(splits, [train_images, val_images, test_images]):
        for image_name in image_set:
            img_path = os.path.join(class_path, image_name)
            # getting the embeding for the photo
            # ide
            embedding = extract_embeddings(img_path)
            if embedding is not None:
                dataset_dict[split].append({
                    'image': img_path,
                    'label': class_name,
                    'embedding': embedding
                })

# Convert the dataset dictionary to Dataset objects
datasets = {
    split: Dataset.from_dict({
        'image': [item['image'] for item in items],
        'label': [item['label'] for item in items],
        'embedding': [item['embedding'] for item in items]
    })
    for split, items in dataset_dict.items()
}

# Create the final DatasetDict
dataset = DatasetDict(datasets)

# Print out the structure of the dataset
print(dataset)


In [ ]:
dataset.save_to_disk("./Dataset")

In [8]:
from datasets import load_dataset

# Load the local dataset
dataset = load_dataset('imagefolder', data_dir=output_dir)

Resolving data files:   0%|          | 0/2037 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/244 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/281 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 2037
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 244
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 281
    })
})

In [ ]:
# Define embedding extraction function
def compute_embeddings(example):
    img = example["image"]  # Load the image
    img = np.array(img)  # Ensure the image is in numpy array format
    
    # Detect face locations
    face_locations = dlib_detector(img)
    
    # If no face is detected, return a zero array
    if len(face_locations) == 0:
        example["embeddings"] = np.zeros(128)  # Assuming embeddings are 128-dimensional
        return example
    
    # Compute embeddings
    face_encodings = encodings(img, face_locations, pose_predictor, face_encoder)
    
    # Store embeddings in the example
    example["embeddings"] = face_encodings.tolist()  # Convert to list for serialization
    return example

# Apply the function to the DatasetDict
# This assumes dataset['train'], dataset['validation'], etc., are present
dataset_with_embeddings = dataset.map(compute_embeddings, batched=False)

# Inspect the updated dataset
print(dataset_with_embeddings)


Parameter 'function'=<function compute_embeddings at 0x000001BFA713FA30> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2037 [00:00<?, ? examples/s]